In [1]:
# 1. Install all system dependencies (with -y to auto-confirm)
!apt-get update
!apt-get install -y xvfb x11-xserver-utils
!apt-get install -y \
    libx11-6 \
    libxau6 \
    libxdmcp6 \
    libxcb1 \
    libxext6 \
    libx11-xcb1 \
    libvulkan1 \
    vulkan-utils \
    libvulkan-dev \
    mesa-vulkan-drivers

# 2. Install Python packages
!pip install --upgrade mani_skill tyro pyvirtualdisplay

# 3. Verify Xvfb is installed
!which Xvfb

# 4. Setup virtual display
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1024, 768))
virtual_display.start()

# Other added
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

0% [Working]

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
x11-xserver-utils is already the newest version (7.7+9build1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.12).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package vulkan-utils
/usr/bin/Xvfb
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show th

In [2]:
import mani_skill.envs
import gymnasium as gym

N = 4
env = gym.make("PickCube-v1",num_envs=N)
print(env.action_space)
print(env.observation_space)

env.reset()

obs, rew, terminated, truncated, info = env.step(env.action_space.sample())


Box(-1.0, 1.0, (4, 8), float32)
Box(-inf, inf, (4, 42), float32)


In [21]:
from mani_skill.vector.wrappers.gymnasium import ManiSkillVectorEnv
from mani_skill.utils.wrappers.record import RecordEpisode
from collections import defaultdict
import torch

env_id = "PushCube-v1"
num_eval_envs = 64
env_kwargs=dict(obs_mode="rgb",render_mode="rgb_array")
eval_envs = gym.make(env_id,num_envs=num_eval_envs,reconfiguration_freq=1,**env_kwargs)
eval_envs = RecordEpisode(
    eval_envs,
    output_dir="random_videos",
    save_trajectory=True,
    trajectory_name="trajectory_{num_eval_envs}",
    save_video=True,
    max_steps_per_video=50
)
eval_envs = ManiSkillVectorEnv(eval_envs,ignore_terminations=True,record_metrics=True)


obs,_ = eval_envs.reset(seed=0)
eval_metrics=defaultdict(list)
for i in range(400):
    action = eval_envs.action_space.sample()

    obs,rew,terminated,truncated,info = eval_envs.step(action)
    # print(f"taken step {i}")
    if truncated.any():
        print(f"step {i}: truncated. metrics recording and env is resetting")
        for k,v in info["final_info"]["episode"].items():
            eval_metrics[k].append(v.float())
for k in eval_metrics.keys():
    print(f"{k}_mean: {torch.mean(torch.stack(eval_metrics[k])).item()}")

2025-01-18 18:56:23,315 - mani_skill  - WARNING - mani_skill is not installed with git.


step 49: truncated. metrics recording and env is resetting
step 99: truncated. metrics recording and env is resetting
step 149: truncated. metrics recording and env is resetting
step 199: truncated. metrics recording and env is resetting
step 249: truncated. metrics recording and env is resetting
step 299: truncated. metrics recording and env is resetting
step 349: truncated. metrics recording and env is resetting
step 399: truncated. metrics recording and env is resetting
success_once_mean: 0.0
return_mean: 4.209468841552734
episode_len_mean: 50.0
reward_mean: 0.08418938517570496
success_at_end_mean: 0.0


## Evaluating Trained PPO

runs are in /runs. model is trained and eval videos are recorded using Maniskill/examples/baselines/ppo